<a href="https://colab.research.google.com/github/Suyash018/kaggle_comp/blob/main/titanic_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
! kaggle competitions download -c titanic

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile

zip_ref = zipfile.ZipFile("titanic.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [ ]:
import pandas as pd

data= pd.read_csv("train.csv")
test= pd.read_csv("test.csv")
test_ids = test["PassengerId"]



In [ ]:
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
data = data.drop(["PassengerId","Name","Ticket"],axis=1)
test=test.drop(["PassengerId","Name","Ticket"],axis=1)


In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Create column transformer (this will help us normalize/preprocess our data)
ct = make_column_transformer(
    (MinMaxScaler(), ["Pclass", "SibSp", "Fare","Age","Parch","Ticket"]), # get all values between 0 and 1
    (OneHotEncoder(handle_unknown="ignore"), ["Sex", "Embarked"])
)

# Create X & y
X = data.drop("Survived", axis=1)
y = data["Survived"]



# Build our train and test sets (use random state to ensure same split as before)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit column transformer on the training data only (doing so on test data would result in data leakage)
ct.fit(X_train)
ct.fit(test)
# Transform training and test data with normalization (MinMaxScalar) and one hot encoding (OneHotEncoder)
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)
test=ct.transform(test)


imputer = SimpleImputer(strategy='mean')
# fit on the dataset
imputer.fit(X_train_normal)
imputer.fit(test)
# transform the dataset
X_train_normal = imputer.transform(X_train_normal)
test=imputer.transform(test)

imputer = SimpleImputer(strategy='mean')
# fit on the dataset
imputer.fit(X_test_normal)
# transform the dataset
X_test_normal = imputer.transform(X_test_normal)

ValueError: ignored

In [ ]:
# Set random seed
import tensorflow as tf
tf.random.set_seed(42)

# Add an extra layer and increase number of units
model = tf.keras.Sequential([
  tf.keras.layers.Dense(100),
  tf.keras.layers.Dense(10, activation=tf.keras.activations.relu),
  tf.keras.layers.Dense(1) 
])

model.compile(
      optimizer=tf.keras.optimizers.Adam(),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=['accuracy'])

model.fit(
    X_train_normal, 
    y_train, 
    validation_data = (X_test_normal, y_test),
    epochs = 100)


NameError: ignored

In [ ]:
import numpy as np
Survived = model.predict(test)
survi= (Survived.flatten()).tolist()

for x in range(len(survi)):
  if survi[x]>=0.5:
    survi[x]=1
  else:
    survi[x]=0

  


In [ ]:
survi


In [ ]:
df = pd.DataFrame({"PassengerId":test_ids.values,
                   "Survived":survi})

In [ ]:
df.to_csv("submission.csv",index=False)

In [ ]:
!kaggle competitions submit -c titanic -f submission.csv -m "Message"